In [1]:
%load_ext autoreload
%autoreload 2
#---------- Library Imports ----------
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import sequential_creator
import glob
from torchmetrics import MeanAbsolutePercentageError as MAPE
import matplotlib as mpl
import re
import json
from tqdm import tqdm
import pytorch_lightning as pl
from torch.utils.data import DataLoader

c:\Users\Lasse\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext autoreload

In [3]:
#---------- Personal python files imports ----------
import load_lettuce_dataset
import data_augmentation
import model_and_training_files
import lettuce_dataset

In [4]:
#---------- Hyperparameters ----------
batch_size = 16
learning_rate = 1e-3

In [5]:
#---------- Other parameters ----------
augmented_dataset_size = 341 # Size of the augmented dataset, so if the original dataset contained 103 images, they would be augmented and made into 2000 images
                              # One thing to note is that if this number is much bigger than the size of the original dataset, then they would most likely end up being duplicates, since there is not that many augmentations implemented at the moment

In [6]:
#---------- Load Lettuce Dataset ----------
rgb_list, depth_list, fresh_weight_list, dry_weight_list = load_lettuce_dataset.load_all_images()

100%|██████████| 341/341 [00:05<00:00, 66.51it/s]


In [7]:
#---------- Augment Lettuce Dataset ----------
depth_img_aug, rgb_imgs_aug, fresh_weight_GT, dry_weight_GT = data_augmentation.augment_data(rgb_images=rgb_list, depth_images=depth_list, fresh_weight_GT=fresh_weight_list, dry_weight_GT=dry_weight_list, amount_of_augmentated_images=augmented_dataset_size)

Augmenting RGB Images


100%|██████████| 341/341 [01:03<00:00,  5.35it/s]


Amount of augmented images 341


In [8]:
#---------- Create data loaders ----------
# Concatenate the depth and rgb images
full_dataset = []
full_dataset_labels = []
for i in range(augmented_dataset_size):
    depth_plus_rgb = np.concatenate((depth_img_aug[i], rgb_imgs_aug[i]), axis=0)
    full_dataset.append(depth_plus_rgb)
    #full_dataset.append([depth_img_aug[i], rgb_imgs_aug[i]])
    full_dataset_labels.append([fresh_weight_GT[i], dry_weight_GT[i]])

print("Size of images in the dataset: ", np.array(full_dataset).shape, " --> (Amount of images, total channels, img_w, img_h)")
print("Size of images in the dataset: ", np.array(full_dataset[0][0]).shape)

# Define the dataset
dataset = lettuce_dataset.LettuceDataset(full_dataset, full_dataset_labels)

# Split the dataset in train, validation and test
splitted_data = lettuce_dataset.data_splittage(augmented_dataset_size, [75, 12.5, 12.5])
print("train, validation, test = ", splitted_data)

train_set, validation_set, test_set = torch.utils.data.random_split(dataset, splitted_data)
#train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [7500, 1250, 1250])

# Create dataloaders for train, validation and test data
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
validation_loader = DataLoader(dataset = validation_set, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = True)

Size of images in the dataset:  (341, 4, 640, 480)  --> (Amount of images, total channels, img_w, img_h)
Size of images in the dataset:  (640, 480)
train, validation, test =  [255, 42, 44]


In [9]:
#---------- Model Definition ----------
# Code modified from Anders Glent Buch elective course Deep Neural Networks
# Tip: dont forget to include activation layer (ReLu)

def build_cnn_layer(in_channels, out_channels, kernal_size=3, stride=1, padding=1, 
                    use_batchnorm=True, pool=False):
    '''
    in_channels: input channels to 2d convolution layer
    out_channels: ouput channels to 2d convolution layer
    kernal_size: kernal size (refer torch conv2d)
    stride: stride (refer torch conv2d)
    padding: padding (refer torch conv2d)  
    use_batchnorm: enable/disable batchnorm (refer torch BatchNorm2d)
    pool: enable/disable pooling (refer torch MaxPool2D)

    should return the built CNN layer
    '''
    return torch.nn.Sequential(
        torch.nn.Conv2d(in_channels, out_channels, kernal_size, stride=stride, padding=padding),
        torch.nn.ReLU(inplace=True),
        torch.nn.BatchNorm2d(out_channels) if use_batchnorm else torch.nn.Identity(),
        torch.nn.MaxPool2d(4, 4) if pool else torch.nn.Identity(),
    )

# Remember to flatten after convolution layers before its passed to linear layers
# Also activation layer for final layers
def build_cnn_model():
    '''
    pool: enable/disable pooling

    should return built CNN model
    '''
    return torch.nn.Sequential(
        # (B, 4, 640, 480) images
        build_cnn_layer(4, 8, stride=1, pool=True),
        build_cnn_layer(8, 16, stride=1, pool=True), 
        build_cnn_layer(16, 32, stride=1, pool=True),
        torch.nn.Flatten(),
        torch.nn.Linear(2240, 1000),
        torch.nn.ReLU(True),
        torch.nn.Linear(1000, 512),
        torch.nn.ReLU(True),
        torch.nn.BatchNorm1d(512), # (B, 512)
        torch.nn.Linear(512, 2), # (B, 2) logits
    )    
    

In [10]:
#---------- Model declaration ----------
model = build_cnn_model()

In [11]:
print(model)

Sequential(
  (0): Sequential(
    (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=2240, out_features=1000, bias

In [17]:
#---------- Training ----------
print("Cuda available = ", torch.cuda.is_available())

# Define the model with Pytorch Lightning
model = model_and_training_files.BiomassModel(CNNmodel=model,
                                                lr=learning_rate,
                                                )

trainer = model_and_training_files.get_trainer()    # gets the trainer (which is a class that takes the model and dataset)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=validation_loader) # Train the model
trainer.test(model, dataloaders=test_loader)

#---

Auto select gpus: [0]


Cuda available =  True


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                        | Params
----------------------------------------------------------
0 | model     | BiomassModel                | 2.8 M 
1 | loss_func | MeanAbsolutePercentageError | 0     
----------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.047    Total estimated model params size (MB)


Epoch 10: 100%|██████████| 19/19 [00:09<00:00,  2.01it/s, loss=0.716, v_num=13]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|██████████| 3/3 [00:02<00:00,  1.36it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.8664084076881409
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.8664084076881409}]

In [18]:
#------- Save the weights ----------
torch.save(model, "saved_models/best_weights_V1.plk") # Saves the regression head

MemoryError: 

In [15]:
model = torch.load("saved_models/best_weights_V1.plk")
#fresh_weight_GT, dry_weight_GT
imgidx = 6
pred = model.prediction(depth_img_aug[imgidx], rgb_imgs_aug[imgidx])
print(pred)
print(fresh_weight_GT[imgidx], dry_weight_GT[imgidx])
loss = MAPE()
true = torch.from_numpy(np.array([[fresh_weight_GT[imgidx], dry_weight_GT[imgidx]]]))
print(loss(pred,true))

RuntimeError: PytorchStreamReader failed locating file data.pkl: file not found